In [2]:
import numpy as np
n_features = 20
term_pred = 'short_term'
data_file_name = 'data_y10_top_20_var'
endpt = 10;
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [3]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [4]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [5]:
snippets_dir = work_dir+ '/code/snippet'
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'

import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [6]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [7]:
data_full

,ID,time,event,C08DIBAG_YEAR,C11MHAGE,C08OTKAG_YEAR,C02AVGDI,C09CGTDY,CL1NTRIG,C08NEPAG_YEAR,...,C08KYSAG_YEAR,MMLVPWD,C40HR,C09SMKYR,MMLAD,C02AVGSY,C18K3PWK,C08KIDNY,DPMEINT,MMLVSSFS
0,100016012504,3689.025,0,0,41,0,72.0,40,147.0,0,...,0,0.650000,59.000000,12,3.970000,114.0,1,1,0.150000,6.030000
1,100023004268,3689.025,0,0,99,0,64.0,0,89.0,0,...,0,0.770000,43.000000,8,3.540000,94.0,12,1,0.130000,3.300000
2,100033323702,3689.025,0,0,99,0,73.0,0,110.0,0,...,0,0.890000,78.000000,0,3.810000,105.0,5,1,0.160000,4.000000
3,100048415270,1169.000,0,0,99,0,68.0,1,49.0,18,...,0,0.770000,78.000000,13,3.320000,93.0,0,2,0.140000,3.960000
4,100056526386,3689.025,0,0,99,0,76.0,0,91.0,0,...,0,1.030000,54.000000,0,4.390000,118.0,0,1,0.180000,3.270000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,416761219907,3689.025,0,0,58,0,63.0,0,84.0,0,...,0,0.850000,70.000000,0,3.430000,98.0,0,1,0.150000,3.730000
5075,416771521620,3689.025,0,0,99,0,82.0,0,38.0,0,...,0,0.690000,56.000000,0,3.130000,111.0,0,1,0.100000,3.200000
5076,416783315386,3689.025,0,0,99,0,64.0,0,47.0,0,...,0,0.821461,75.000000,3,3.425875,109.0,0,1,0.128792,3.209413
5077,416796224310,3689.025,0,0,99,0,69.0,0,40.0,0,...,0,0.890000,68.809628,12,4.640000,108.0,0,1,0.080000,4.670000


In [8]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/all_training_ID_outerloop_cohort_0_10.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [9]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,18,19
0,100016012504,0,3689.025,-0.085568,-5.899121,-0.083231,0.281825,5.152672,0.999235,-0.056427,...,-0.083403,-1.558528,-0.782068,1.474712,1.025752,0.568188,0.174188,-0.157143,0.813713,1.330147
1,100023004268,0,3689.025,-0.085568,0.201353,-0.083231,-0.569733,-0.444710,0.121282,-0.056427,...,-0.083403,-0.619187,-2.467120,0.778302,0.019936,-1.297814,5.656853,-0.157143,0.099845,-0.081653
2,100033323702,0,3689.025,-0.085568,0.201353,-0.083231,0.388270,-0.444710,0.439162,-0.056427,...,-0.083403,0.320154,1.218932,-0.614518,0.651495,-0.271513,2.167885,-0.157143,1.170647,0.280347
3,100048415270,0,1169.000,-0.085568,0.201353,-0.083231,-0.143954,-0.304776,-0.484202,17.489992,...,-0.083403,-0.619187,1.218932,1.648814,-0.494667,-1.391114,-0.324236,2.360618,0.456779,0.259661
4,100056526386,0,3689.025,-0.085568,0.201353,-0.083231,0.707604,-0.444710,0.151557,-0.056427,...,-0.083403,1.416052,-1.308647,-0.614518,2.008177,0.941388,-0.324236,-0.157143,1.884515,-0.097168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,416761219907,0,3689.025,-0.085568,-4.111051,-0.083231,-0.676177,-0.444710,0.045597,-0.056427,...,-0.083403,0.007041,0.376406,-0.614518,-0.237366,-0.924613,-0.324236,-0.157143,0.813713,0.140718
5075,416771521620,0,3689.025,-0.085568,0.201353,-0.083231,1.346272,-0.444710,-0.650710,-0.056427,...,-0.083403,-1.245414,-1.098015,-0.614518,-0.939097,0.288288,-0.324236,-0.157143,-0.970956,-0.133368
5076,416783315386,0,3689.025,-0.085568,0.201353,-0.083231,-0.569733,-0.444710,-0.514476,-0.056427,...,-0.083403,-0.216356,0.902984,-0.092210,-0.247015,0.101688,-0.324236,-0.157143,0.056723,-0.128500
5077,416796224310,0,3689.025,-0.085568,0.201353,-0.083231,-0.037509,-0.444710,-0.620436,-0.056427,...,-0.083403,0.320154,0.251041,1.474712,2.592953,0.008387,-0.324236,-0.157143,-1.684824,0.626832


In [10]:
data_full = data_full

In [11]:
# decide on number of dicrete times:
#halflife=13.*365.25


#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [12]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features


def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [13]:
# Outerloop:
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    # l2_final=0.1

    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)






FOLD 0:

Train C-index fold 1 :
0.8459712425393381
Test C-index fold 1 :
0.7589065650907462

FOLD 1:

Train C-index fold 2 :
0.862374785346308
Test C-index fold 2 :
0.6613404349755881

FOLD 2:

Train C-index fold 3 :
0.8630564159677027
Test C-index fold 3 :
0.7107818725099602

FOLD 3:

Train C-index fold 4 :
0.8217821074524884
Test C-index fold 4 :
0.9204279505182213

FOLD 4:

Train C-index fold 5 :
0.8461612723294097
Test C-index fold 5 :
0.7038273253226525

FOLD 5:

Train C-index fold 6 :
0.8509089607787003
Test C-index fold 6 :
0.7244207959206297

FOLD 6:

Train C-index fold 7 :
0.8413690071981529
Test C-index fold 7 :
0.7154634581105169

FOLD 7:

Train C-index fold 8 :
0.8273211097138172
Test C-index fold 8 :
0.8107819756495545

FOLD 8:

Train C-index fold 9 :
0.845857542039177
Test C-index fold 9 :
0.7353695240218482

FOLD 9:

Train C-index fold 10 :
0.837973959078552
Test C-index fold 10 :
0.8533362939935606

FOLD 10:

Train C-index fold 11 :
0.848336041575696
Test C-index fold 